# Use the manual count parser prototype with OTAnalytics

> Note: The base path is set in `.vscode/settings.json` to workspace path. Otherwise Jupyter can't import OTAnalytics.

## Import the prototype modules

Before working with events, you need to import the event_processor to convert the events to a shaped data frame.

In [1]:
# Import libraries and modules
# OTAnalytics modules
from OTAnalytics.plugin_prototypes.manual_count_parser.manual_count_parser import (
    ExcelCountParser,
)
from OTAnalytics.plugin_prototypes.mc_miovision_parser.mc_miovision_parser import (
    McMioParser,
)
from OTAnalytics.plugin_prototypes.event_parser.event_parser import EventParser
from OTAnalytics.plugin_prototypes.counter.counter import Counter
import pandas as pd
import plotly.express as px


%load_ext autoreload
%autoreload 2

## Set config variables

In the current version, confiogs for prototypes are stored in dictionries within the Jupyter notebook. In a future version, the will likely be stored in a config file.

The path to the eventlists can be a path to a single event list or a folder (all otevent files within this folder will be imported).

In [3]:
# Set env parameters and path
CONFIG = {
    "EXCEL_PATH": "/Volumes/platomo data/Produkte/OpenTrafficCam/Videos andere Cams/Modus Consult Tests/2023-04-25_Testdaten/Max (PLATOMO@sebastian.buck)/K3_Forster Str-BNeumann-Str_24h_Kfz-SV_Di.xlsm",
    "TIME_FORMAT": "%d.%m%.%y %H:%M Uhr",
    "FILTER_CLASS": [],
    "FILTER_SECTION": [],
    "EVENTLIST_PATH": "/Volumes/platomo data/Projekte/012 Videoauswertung LBV-SH B57/videos",
    "SECTIONSLIST_PATH": "/Volumes/platomo data/Projekte/012 Videoauswertung LBV-SH B57/videos/sectionlist_SH_new.otflow",
    "FROM_TIME": "2023-03-28 00:00:00",
    "TO_TIME": "2023-03-29 00:00:00",
    "INTERVAL_LENGTH_MIN": 15,  # in minutes
    "DIRECTION_NAMES": {
        "first_to_last_section": "in",
        "last_to_first_section": "out",
    },
}

## Define flows

In [4]:
classes_street = [
    "bicyclist",
    "bicyclist_with_trailer",
    "cargobike_driver",
    "car",
    "car_with_trailer",
    "bus",
    "motorcyclist",
    "delivery_van_with_trailer",
    "delivery_van",
    "private_van",
    "private_van_with_trailer",
    "truck",
    "truck_with_trailer",
    "truck_with_semitrailer",
]

flow_names = {
    "Strom 01": {
        "classes": classes_street,
        "from": ["B75_West_S1-2-3_start_S4_end", "B75_West_S1-2_start_S4-8-12_end"],
        "to": [
            "Kroegersweg_S2-10-11-12_start_S1-5-9_end",
            "Kroegersweg_S2-10-11-12_start_S1-5-8_end",
        ],
    },
    "Strom 02": {
        "classes": classes_street,
        "from": [
            "B75_West_S1-2-3_start_S4_end",
            "B75_Ost_S2_start",
            "B75_West_S1-2_start_S4-8-12_end",
            "Kroegersweg_S2-10-11-12_start_S1-5-9_end",
            "Kroegersweg_S2-10-11-12_start_S1-5-8_end",
        ],
        "to": ["B75_Ost_S2-6_end_1", "B75_Ost_S2-6_end_2"],
    },
    "Strom 03": {
        "classes": classes_street,
        "from": ["B75_West_S1-2-3_start_S4_end"],
        "to": [
            "Rothenmoor_S4-5-6_start_S3-7-11_end",
            "Rothenmoor_S4-5-6_start_S3-7-11-Q2S_end",
        ],
    },
    "Strom 04": {
        "classes": classes_street,
        "from": [
            "Rothenmoor_S4-5-6_start_S3-7-11_end",
            "Rothenmoor_S4-5-6_start_S3-7-11-Q2S_end",
        ],
        "to": [
            "B75_West_S1-2-3_start_S4_end",
            "B75_West_S1-2_start_S4-8-12_end",
            "B75_West_S4-8-12_end",
        ],
    },
    "Strom 05": {
        "classes": classes_street,
        "from": [
            "Rothenmoor_S4-5-6_start_S3-7-11_end",
            "Rothenmoor_S4-5-6_start_S3-7-11-Q2S_end",
        ],
        "to": [
            "Kroegersweg_S2-10-11-12_start_S1-5-9_end",
            "Kroegersweg_S2-10-11-12_start_S1-5-8_end",
            "B75_Ost_S2-7-8-9_start_S5_end",
        ],
    },
    "Strom 06": {
        "classes": classes_street,
        "from": [
            "Rothenmoor_S4-5-6_start_S3-7-11_end",
            "Rothenmoor_S4-5-6_start_S3-7-11-Q2S_end",
        ],
        "to": ["B75_Ost_S2-6_end_1", "B75_Ost_S2-6_end_2"],
    },
    "Strom 07": {
        "classes": classes_street,
        "from": ["B75_Ost_S2-7-8-9_start_S5_end"],
        "to": [
            "Rothenmoor_S4-5-6_start_S3-7-11_end",
            "Rothenmoor_S4-5-6_start_S3-7-11-Q2S_end",
        ],
    },
    "Strom 08": {
        "classes": classes_street,
        "from": ["B75_Ost_S2-7-8-9_start_S5_end"],
        "to": [
            "Kroegersweg_S2-10-11-12_start_S1-5-8_end",
            "B75_West_S4-8-12_end",
            "B75_West_S1-2_start_S4-8-12_end",
        ],
    },
    "Strom 09": {
        "classes": classes_street,
        "from": ["B75_Ost_S2-7-8-9_start_S5_end"],
        "to": ["Kroegersweg_S2-10-11-12_start_S1-5-9_end"],
    },
    "Strom 10": {
        "classes": classes_street,
        "from": [
            "Kroegersweg_S2-10-11-12_start_S1-5-9_end",
        ],
        "to": ["B75_Ost_S2-6_end_1", "B75_Ost_S2-6_end_2"],
    },
    "Strom 11": {
        "classes": classes_street,
        "from": [
            "Kroegersweg_S2-10-11-12_start_S1-5-9_end",
            "Kroegersweg_S2-10-11-12_start_S1-5-8_end",
        ],
        "to": [
            "Rothenmoor_S4-5-6_start_S3-7-11_end",
            "Rothenmoor_S4-5-6_start_S3-7-11-Q2S_end",
        ],
    },
    "Strom 12": {
        "classes": classes_street,
        "from": [
            "Kroegersweg_S2-10-11-12_start_S1-5-9_end",
            "Kroegersweg_S2-10-11-12_start_S1-5-8_end",
        ],
        "to": ["B75_West_S4-8-12_end", "B75_West_S1-2_start_S4-8-12_end"],
    }
}

mode_mapper = {
    "bicyclist": "RAD",
    "bicyclist_with_trailer": "RAD",
    "cargobike_driver": "RAD",
    "car": "PKW & LFW",
    "car_with_trailer": "PKW & LFW",
    "bus": "BUS",
    "motorcyclist": "KRAD",
    "delivery_van": "PKW & LFW",
    "delivery_van_with_trailer": "PKW & LFW",
    "private_van": "PKW & LFW",
    "private_van_with_trailer": "PKW & LFW",
    "truck": "LKW ab 3,5t",
    "truck_with_trailer": "LZ/SFZ",
    "truck_with_semitrailer": "LZ/SFZ",
}

## Import Events

In [35]:
event_processor = EventParser(CONFIG)
events = event_processor.process_events()
"""events.to_csv(
    "/Users/michaelheilig/platomo/Projekte/012 Videoauswertung LBV-SH B75/work/Auswertung/Detection/B75_SH_events.csv",
    index=False,
)"""

'events.to_csv(\n    "/Users/michaelheilig/platomo/Projekte/012 Videoauswertung LBV-SH B75/work/Auswertung/Detection/B75_SH_events.csv",\n    index=False,\n)'

## Create Flow Table

In [34]:
flow_processor = Counter(CONFIG, events)

filter_sections = []
filter_classes = []

flows = flow_processor.get_flows(filter_sections, filter_classes)
flow_table = flow_processor.create_flow_table(filter_sections, filter_classes)
flow_table = flow_table[flow_table["from_section"] != flow_table["to_section"]]
flow_table = flow_processor.convert_flow_table(
    flow_table, flow_names, mode_mapper, aggregated=True
)

NameError: name 'events' is not defined

## Import Excel File

In [33]:
parser = McMioParser(
    id_dict=id_dict,
    CONFIG=CONFIG,
    access_roads={"Zufahrt1": "Bahnhofstraße (N)", 
                "Zufahrt3": "Bahnhofstraße (O)", 
                "Zufahrt5": "Bahnhofstraße (S)" , 
                "Zufahrt7": "Bismarckstraße"}
)
excel_table_processed = parser.excel_parser(aggregate=True)

## Compare results

In [ ]:
compare_flows = (
    pd.merge(
        flow_table,
        excel_table_processed,
        on=["Datum", "Uhrzeit", "Strom-Bezeichnung", "Fzg-Typ"],
        how="left",
        suffixes=["_det", "_real"],
    )
    .fillna(0)
    .rename({"Anzahl_det": "Videodetektion", "Anzahl_real": "Manuelle Zählung"}, axis=1)
)

## Plot comparison

In [ ]:
flow_plot_data = pd.melt(
    compare_flows,
    id_vars=["Datum", "Uhrzeit", "Strom-Bezeichnung", "Fzg-Typ"],
    value_vars=["Videodetektion", "Manuelle Zählung"],
    value_name="Anzahl",
    var_name="Quelle",
)

In [ ]:
plot_data = flow_plot_data
fig = px.bar(
    plot_data,
    x="Uhrzeit",
    y="Anzahl",
    color="Quelle",
    category_orders={"Quelle": ["Manuelle Zählung", "Videodetektion"]},
    barmode="group",
    facet_row="Strom-Bezeichnung",
    height=len(plot_data["Strom-Bezeichnung"].unique()) * 400,
    facet_row_spacing=0.02,
    title="Vergleich Videodetektion - manuelle Zählung (je Strom)",
    custom_data=["Quelle", "Fzg-Typ"],
)
fig.update_xaxes(title="Uhrzeit", visible=True, showticklabels=True)
fig.update_traces(
    hovertemplate="Time:%{x}<br>Value:%{value}<br>Fzg-Typ:%{customdata[1]}"
)
fig.write_html(
    "/Users/michaelheilig/platomo/Projekte/012 Videoauswertung LBV-SH B75/work/Auswertung/Vergleich_Zählung_B75_Ströme.html"
)
fig.show()

In [ ]:
plot_data = (
    flow_plot_data.groupby(["Datum", "Uhrzeit", "Fzg-Typ", "Quelle"])
    .sum()
    .reset_index()
)
fig = px.bar(
    plot_data,
    x="Uhrzeit",
    y="Anzahl",
    color="Quelle",
    category_orders={"Quelle": ["Manuelle Zählung", "Videodetektion"]},
    barmode="group",
    # facet_row="Strom-Bezeichnung",
    # height=len(plot_data["Strom-Bezeichnung"].unique()) * 400,
    title="Vergleich Videodetektion - manuelle Zählung (alle Ströme)",
    custom_data=["Quelle", "Fzg-Typ"],
)
fig.update_xaxes(title="Uhrzeit", visible=True, showticklabels=True)
fig.update_traces(
    hovertemplate="Time:%{x}<br>Value:%{value}<br>Fzg-Typ:%{customdata[1]}"
)
fig.write_html(
    "/Users/michaelheilig/platomo/Projekte/012 Videoauswertung LBV-SH B75/work/Auswertung/Vergleich_Zählung_B75_gesamt.html"
)

fig.show()

## Export Files

In [ ]:
parser = ExcelCountParser(
    id_dict=id_dict,
    CONFIG=CONFIG,
)
excel_events = parser.excel_parser(aggregate=False).drop("Anzahl", axis=1)
excel_events = excel_events[
    (
        (excel_events["Uhrzeit"].str.match("0[0-7]:\d\d:*"))
        | (excel_events["Uhrzeit"].str.match("[1][8-9]:\d\d:*"))
        | (excel_events["Uhrzeit"].str.match("[2][0-4]:\d\d:*"))
    )
    & (excel_events["Strom-Bezeichnung"] != "SIDEWALK")
]
flow_processor = Counter(CONFIG, events)

flows_export = flow_processor.convert_flow_table(
    flows, flow_names, mode_mapper, aggregated=False
)
flows_export_day = flows_export[
    (flows_export["Uhrzeit"].str.match("0[8-9]:\d\d:*"))
    | (flows_export["Uhrzeit"].str.match("[1][0-7]:\d\d:*"))
    | (flows_export["Strom-Bezeichnung"].str.match("Strom\sQ\s2*"))
]
flows_export_merged = (
    pd.concat([flows_export_day, excel_events])
    .sort_values(["Datum", "Uhrzeit"])
    .reset_index(drop=True)
)

In [ ]:
parser = ExcelCountParser(
    id_dict=id_dict,
    CONFIG=CONFIG,
)
excel_table = parser.excel_parser(aggregate=True)
excel_table = pd.merge(
    flow_table.drop("Anzahl", axis=1),
    excel_table,
    on=["Datum", "Uhrzeit", "Strom-Bezeichnung", "Fzg-Typ"],
    how="outer",
).fillna(0)
excel_table = excel_table[
    (
        (excel_table["Uhrzeit"].str.match("0[0-7]:\d\d:*"))
        | (excel_table["Uhrzeit"].str.match("[1][8-9]:\d\d:*"))
        | (excel_table["Uhrzeit"].str.match("[2][0-4]:\d\d:*"))
    )
    & (excel_table["Strom-Bezeichnung"] != "SIDEWALK")
]


flow_processor = Counter(CONFIG, events)

flows_table_export_day = flow_table[
    (flow_table["Uhrzeit"].str.match("0[8-9]:\d\d:*"))
    | (flow_table["Uhrzeit"].str.match("[1][0-7]:\d\d:*"))
    | (flow_table["Strom-Bezeichnung"].str.match("Strom\sQ\s2*"))
]
flow_table_export_merged = (
    pd.concat([flows_table_export_day, excel_table])
    .sort_values(["Datum", "Uhrzeit"])
    .reset_index(drop=True)
)

In [ ]:
print(sum(excel_table["Anzahl"]))
print(len(excel_events))

In [ ]:
print(len(flows_export))
print(sum(flow_table["Anzahl"]))

In [ ]:
print(len(flows_export_merged))
print(sum(flow_table_export_merged["Anzahl"]))

In [ ]:
flows_export.to_csv(
    "/Users/michaelheilig/platomo/Projekte/012 Videoauswertung LBV-SH B75/work/Auswertung/Zählung_B75_einzel.csv",
    index=False,
)
flows_export_merged.to_csv(
    "/Users/michaelheilig/platomo/Projekte/012 Videoauswertung LBV-SH B75/work/Auswertung/Zählung_B75_einzel_mit-manuell.csv",
    index=False,
)

flow_table.to_csv(
    "/Users/michaelheilig/platomo/Projekte/012 Videoauswertung LBV-SH B75/work/Auswertung/Zählung_B75_agg15min.csv",
    index=False,
)
flow_table_export_merged.to_csv(
    "/Users/michaelheilig/platomo/Projekte/012 Videoauswertung LBV-SH B75/work/Auswertung/Zählung_B75_agg15min_mit-manuell.csv",
    index=False,
)

In [ ]:
flows_export.to_excel(
    "/Users/michaelheilig/platomo/Projekte/012 Videoauswertung LBV-SH B75/work/Auswertung/Zählung_B75_einzel.xlsx",
    index=False,
)
flows_export_merged.to_excel(
    "/Users/michaelheilig/platomo/Projekte/012 Videoauswertung LBV-SH B75/work/Auswertung/Zählung_B75_einzel_mit-manuell.xlsx",
    index=False,
)

flow_table.to_excel(
    "/Users/michaelheilig/platomo/Projekte/012 Videoauswertung LBV-SH B75/work/Auswertung/Zählung_B75_agg15min.xlsx",
    index=False,
)
flow_table_export_merged.to_excel(
    "/Users/michaelheilig/platomo/Projekte/012 Videoauswertung LBV-SH B75/work/Auswertung/Zählung_B75_agg15min_mit-manuell.xlsx",
    index=False,
)